## `GPUDrive` simulator concepts

In this notebook, we demonstrate how to work with the `GPUDrive` simulator and access its basic attributes in Python. The simulator, written in C++, is built on top of the [Madrona Engine](https://madrona-engine.github.io/).

In [65]:
import gpudrive
import torch 
import os
os.chdir('..')

### Summary

- `GPUDrive` simulations are discretized traffic scenarios. A scenario is a constructed snapshot of traffic situation at a particular timepoint.
- The state of the vehicle of focus is referred to as the **ego state**. Each vehicle has their own partial view of the traffic scene; and a visible state is constructed by parameterizing the view distance of the driver. The **action** for each vehicle is a (1, 3) tuple with the acceleration, steering and head angle of the vehicle.
- The `step()` method advances the simulation with a desired step size. By default, the dynamics of vehicles are driven by a kinematic bicycle model. If a vehicle is not controlled (that is, we do not give it actions), its position, heading, and speed will be updated according to a the human expert demonstrations.


### Instantiating a sim object with default parameters

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
sim = gpudrive.SimManager(
    exec_mode=gpudrive.madrona.ExecMode.CUDA if device == "cuda" else gpudrive.madrona.ExecMode.CPU, 
    gpu_id=0, 
    num_worlds=1, # The number of parallel environments
    json_path='../example_data', # Path to data files, must be structured as above
    params = gpudrive.Parameters() # Environment parameters
)

The simulator provides the following functions:
- `reset(world_idx)` resets a specific world or environment at the given index.

In [28]:
sim.reset(0)

- `step()` advances the dynamics of all worlds.

In [29]:
sim.step() 

### Exporting tensors

To retrieve a tensor from the simulator, call the specific `tensor()` method, followed by either `to_torch()` or `to_jax()`.

For example, here is how to access the ego state, or self-observation tensor:

In [31]:
observation_tensor = sim.self_observation_tensor().to_torch()

observation_tensor.shape, observation_tensor.device

(torch.Size([1, 128, 6]), device(type='cpu'))

Or alternatively:

In [36]:
observation_tensor_jax = sim.self_observation_tensor().to_jax()

observation_tensor_jax.shape, observation_tensor_jax.devices()

((1, 128, 6), {CpuDevice(id=0)})

---

> #### ✏️ All these extractions are handled for you in the `gym` environments introduced in the next tutorial.

---

Here are all available tensor exports and methods on the sim object:

In [39]:
for attr in dir(sim):
    if not attr.startswith('_'):
        print(attr)

absolute_self_observation_tensor
action_tensor
agent_roadmap_tensor
controlled_state_tensor
depth_tensor
done_tensor
expert_trajectory_tensor
info_tensor
lidar_tensor
map_observation_tensor
partner_observations_tensor
reset
reset_tensor
response_type_tensor
reward_tensor
rgb_tensor
self_observation_tensor
shape_tensor
step
steps_remaining_tensor
valid_state_tensor


### Inspect valid and controlled agents

To check the number of agents and road points in each world, you can use the `shape_tensor`.

The shape tensor is a 2D tensor where the first dimension represents the number of worlds, and the second dimension represents the shape of each world.

In [67]:
shape_tensor = sim.shape_tensor().to_jax()
print(f'Shape tensor has a shape of (Num Worlds, 2): {shape_tensor.shape}')

for world_idx in range(shape_tensor.shape[0]):
    print(f'World {world_idx} has {shape_tensor[world_idx][0]} VALID agents and {shape_tensor[world_idx][1]} VALID road objects')

Shape tensor has a shape of (Num Worlds, 2): (1, 2)
World 0 has 10 VALID agents and 3195 VALID road objects


The number of **valid** agents indicates the number of controllable agents (vehicles). Some vehicles or bicycles may be initialized in incorrect positions or remain static; these are marked as **invalid** and cannot be controlled.

The sim comes with a mask that indicates which agents can be controlled. Entries are `1` for agents that can be controlled, and `0` otherwise.

In [68]:
controlled_state_tensor = sim.controlled_state_tensor().to_torch()
print("Controlled state tensor has a shape of (num_worlds, max_num_objects, 1): ", controlled_state_tensor.shape)

Controlled state tensor has a shape of (num_worlds, max_num_objects, 1):  torch.Size([1, 128, 1])


In [69]:
# We can control 3 agents in this world
controlled_state_tensor.squeeze()

tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [70]:
controlled_state_tensor.sum().item()

3

### Actions

The action tensor stores the current actions for all agents across all worlds:

In [75]:
action_tensor = sim.action_tensor().to_torch()
print(f'Action tensor has a shape of (num_worlds, max_num_objects, 3): {action_tensor.shape}')

Action tensor has a shape of (num_worlds, max_num_objects, 3): torch.Size([1, 128, 3])


To set the actions for all controlled agents, we use the `copy_()` method:

In [81]:
actions_tensor = sim.action_tensor().to_torch()

actions = torch.full(actions_tensor.shape, 1.0)
actions_tensor.copy_(actions)

print(f'Actions tensor after setting all actions to 1: {actions_tensor[0][0]}')

# Call step() to apply the actions
sim.step()

Actions tensor after setting all actions to 1: tensor([1., 1., 1.])


### Inspecting the simulator settings

In [83]:
params = gpudrive.Parameters()

print('Parameters:')
for attr in dir(params):
    if not attr.startswith('__'):
        value = getattr(params, attr)
        print(f"{attr:20}: {value}")
        if attr == 'rewardParams':
            print('Reward parameters:')
            reward_params = getattr(params, attr)
            for attr2 in dir(reward_params):
                if not attr2.startswith('__'):
                    value2 = getattr(reward_params, attr2)
                    print(f"    {attr2:18}: {value2}")


Parameters:
IgnoreNonVehicles   : False
collisionBehaviour  : gpudrive.CollisionBehaviour.AgentStop
datasetInitOptions  : gpudrive.DatasetInitOptions.FirstN
disableClassicalObs : False
enableLidar         : False
initOnlyValidAgentsAtFirstStep : True
isStaticAgentControlled: False
maxNumControlledVehicles: 10000
observationRadius   : 0.0
polylineReductionThreshold: 0.0
rewardParams        : <gpudrive.RewardParams object at 0x7f0393f610b0>
Reward parameters:
    distanceToExpertThreshold: 0.0
    distanceToGoalThreshold: 0.0
    rewardType        : gpudrive.RewardType.DistanceBased
roadObservationAlgorithm: gpudrive.FindRoadObservationsWith.KNearestEntitiesWithRadiusFiltering
useWayMaxModel      : False


### Setting the simulator parameters

To set the parameters of the simulator, fill in the values for each attribute of the parameter object as below. This allows you to customize the simulation settings.

The params object can be passed to the sim constructor like this:

```Python
sim = gpudrive.SimManager(
    ...
    params=params 
)
```

See our [README](https://github.com/Emerge-Lab/gpudrive/tree/main?tab=readme-ov-file#configuring-the-sim) for the full documentation.

In [84]:
reward_params = gpudrive.RewardParams()
reward_params.rewardType = gpudrive.RewardType.DistanceBased
reward_params.distanceToGoalThreshold = 1.0
reward_params.distanceToExpertThreshold = 1.0 

# Initialize Parameters
params = gpudrive.Parameters()
params.polylineReductionThreshold = 1.0
params.observationRadius = 100.0
params.datasetInitOptions = gpudrive.DatasetInitOptions.RandomN
params.collisionBehaviour = gpudrive.CollisionBehaviour.Ignore
params.maxNumControlledVehicles = 10
params.rewardParams = reward_params

### Running an episode of the sim

Putting everything together, the full interaction loop looks like this:

In [85]:
num_worlds = 1

for i in range(num_worlds):
    sim.reset(i)

actions_shape = sim.action_tensor().to_torch().shape
dones = sim.done_tensor().to_torch()
while not torch.all(sim.done_tensor().to_torch()):
    obs, rews, dones = sim.self_observation_tensor().to_torch(), sim.reward_tensor().to_torch(), sim.done_tensor().to_torch()
    actions = torch.rand(actions_shape)
    sim.action_tensor().to_torch().copy_(actions)
    sim.step()